In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-capec-multilabel/capec_mutilabel_new.csv
/kaggle/input/data-capec-multilabel/data_capec_multilabel.csv


In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import keras
from keras.models import Model, load_model
# from keras.layers import Input, Embedding, Dense, Dropout, Flatten, Conv1D, MaxPool1D, Add, Reshape, LayerNormalization, Concatenate, GlobalMaxPooling1D


import tensorflow as tf
from tensorflow.keras.layers import Bidirectional,Embedding ,Dropout, Dense, Concatenate, Input, Flatten,Conv1D, MaxPool1D, Reshape, LayerNormalization, Concatenate, GlobalMaxPooling1D, BatchNormalization
from tensorflow.keras.models import Model

# from keras.utils import plot_model, to_categorical, np_utils
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from keras import optimizers

2024-04-10 10:34:48.607694: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 10:34:48.607839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 10:34:48.715830: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df_1 = pd.read_csv("/kaggle/input/data-capec-multilabel/data_capec_multilabel.csv")

/tmp/ipykernel_34/1063578078.py:1: DtypeWarning: Columns (9,11,12,13,14,15,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1 = pd.read_csv("/kaggle/input/data-capec-multilabel/data_capec_multilabel.csv")


In [4]:
df_1.columns

Index(['timestamp', 'src_ip', 'src_port', 'dst_ip', 'dst_port',
       'request_http_method', 'request_http_request', 'request_http_protocol',
       'request_user_agent', 'request_referer', 'request_host',
       'request_origin', 'request_cookie', 'request_content_type',
       'request_accept', 'request_accept_language', 'request_accept_encoding',
       'request_do_not_track', 'request_connection', 'request_body',
       'response_http_protocol', 'response_http_status_code',
       'response_http_status_message', 'response_content_length',
       '000 - Normal', '272 - Protocol Manipulation', '242 - Code Injection',
       '88 - OS Command Injection', '126 - Path Traversal',
       '66 - SQL Injection', '16 - Dictionary-based Password Attack',
       '310 - Scanning for Vulnerable Software',
       '153 - Input Data Manipulation', '248 - Command Injection',
       '274 - HTTP Verb Tampering', '194 - Fake the Source of Data',
       '34 - HTTP Response Splitting', '33 - HTTP Request

In [5]:
from urllib.parse import unquote_plus

def decode_column(df, column_name):
    """
    Decode values in the specified column of a DataFrame.
    
    Parameters:
        df (pandas.DataFrame): The DataFrame.
        column_name (str): The name of the column to decode.
        
    Returns:
        pandas.DataFrame: A copy of the DataFrame with the specified column decoded.
    """
    def safe_decode(value):
        if isinstance(value, str):  # Kiểm tra nếu value là một chuỗi
            return unquote_plus(value)
        else:
            return value
    
    decoded_values = df[column_name].apply(safe_decode)
    df_decoded = df.copy()
    df_decoded[column_name] = decoded_values
    return df_decoded

In [6]:
df_1 = decode_column(df_1,"request_body")

In [7]:
print(df_1["request_body"])

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
907810    NaN
907811    NaN
907812    NaN
907813    NaN
907814    NaN
Name: request_body, Length: 907815, dtype: object


In [8]:
print(df_1["request_http_request"])


0                                                         /
1         /blog/index.php/2020/04/04/voluptatum-reprehen...
2                                      /blog/xmlrpc.php?rsd
3                                                         /
4         /blog/index.php/2020/04/04/nihil-tenetur-et-ad...
                                ...                        
907810                                                    /
907811                                                    /
907812                                                    /
907813                                                    /
907814                                                    /
Name: request_http_request, Length: 907815, dtype: object


In [9]:
# df_1 = df_1[~pd.isna(df_1["request_body"])]
# df_1["request_body"]

In [10]:
# df_filtered = df_filtered[~pd.isna(df_filtered["request_body"])]
# df_filtered["request_body"]

In [11]:
#df_1_atk_request = df_1_atk["request_http_request"] + df_1_atk["request_http_request"]

df_1['subpath'] = df_1['request_http_request']+df_1['request_body'].astype(str).apply(lambda x: "" if x == "nan" else '?'+ x)

In [12]:
df_1['subpath']

0                                                         /
1         /blog/index.php/2020/04/04/voluptatum-reprehen...
2                                      /blog/xmlrpc.php?rsd
3                                                         /
4         /blog/index.php/2020/04/04/nihil-tenetur-et-ad...
                                ...                        
907810                                                    /
907811                                                    /
907812                                                    /
907813                                                    /
907814                                                    /
Name: subpath, Length: 907815, dtype: object

In [13]:
df_new = df_1['request_http_request']
df_new = df_1[['request_http_request', 'request_body', 'subpath', '000 - Normal']].copy()
df_new = df_new[df_new['subpath'] != '/'].copy()
df_new = df_new.rename(columns={"000 - Normal": "label"})
df_new["label"].value_counts()

label
0    382299
1    226509
Name: count, dtype: int64

In [14]:
# Training : Validation : Test = 7.5 : 1.5 : 1.0
from sklearn.model_selection import train_test_split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
X_train, X_test, y_train, y_test = train_test_split(df_new['subpath'], df_new['label'], test_size=1 - train_ratio, random_state=42)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42) 


print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)

print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)

print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

X_train shape (456606,)
y_train shape (456606,)
X_val shape (91321,)
y_val shape (91321,)
X_test shape (60881,)
y_test shape (60881,)


In [15]:
print(X_train)

527308    /blog/wp-content/uploads/2020/04/d2ac875b-15d8...
371309    /blog/index.php+or+0+in+%28select+sleep%2815%2...
359220    /blog/wp-includes/%2B/underscore.min.js?ver=1.8.3
306847    /blog/https%3A%2F%2F8501779237819759495.owasp....
259982    /blog/index.php/my-account%2522%253Bcat%2B%252...
                                ...                        
111061    /blog/0W45pz4p/sample-page/ut-aspernatur-est-d...
259974    /blog/index.php%2527%253Bcat%2B%252Fetc%252Fpa...
366661             /blog/index.php/author/sipes-nolan/feed/
132726    /blog/index.php/2020/04/04XYZABCDEFGHIJ/nihil-...
122752    /blog/index.php/2020/03/27/qui-ratione-maxime-...
Name: subpath, Length: 456606, dtype: object


In [16]:
import urllib.parse

def url_decode(encoded_URL):
    """
    Decode the URL encoded string.
    
    Parameters:
        encoded_URL (str): The URL encoded string.
        
    Returns:
        str: The decoded URL string.
    """
    if isinstance(encoded_URL, str):  # Kiểm tra nếu tham số là một chuỗi
        if encoded_URL != "/":  # Kiểm tra nếu giá trị không phải là "/"
            decoded_url = urllib.parse.unquote(encoded_URL)
            return decoded_url
        else:
            return encoded_URL  # Trả về giá trị gốc nếu là "/"
    else:
        # Trả về None hoặc thông báo lỗi tùy vào yêu cầu cụ thể của bạn
        return None  # Hoặc raise TypeError("Invalid input type: expected str, got {type(encoded_URL)}")


In [17]:
# df_filtered = df_1[~pd.isna(df_1["request_body"])]
# df_filtered = decode_column(df_new,"request_body")

In [18]:
# df_filtered = df_filtered[~pd.isna(df_filtered["request_body"])]
# df_filtered["request_body"]

In [19]:
def load_data(urls, max_length=1000):
    for url in urls:
        if url != '/':  # Kiểm tra nếu giá trị không phải là '/'
            try:
                url_lower = str(url).lower()
            except AttributeError:  # Bắt ngoại lệ nếu giá trị không thể chuyển đổi thành chuỗi
                print(f"Skipping lowercase conversion for value: {url}")
                url_lower = url
        else:
            url_lower = url  # Giữ nguyên giá trị nếu là '/'
    url_list = []
    for url in urls:
        # url decode
        decoded_url = url_decode(url)
        # unicode encode
        encoded_url = [ord(x) for x in str(decoded_url).strip()]
        encoded_url = encoded_url[:max_length]
        url_len = len(encoded_url)
        if url_len < max_length:
            # zero padding
            encoded_url += ([0] * (max_length - url_len))
        url_list.append((encoded_url))
    # convert to numpy array
    url_list = np.array(url_list)
    return url_list

In [20]:
train_data = load_data(X_train)
val_data = load_data(X_val)
test_data = load_data(X_test)

In [21]:
def create_label(labels):
  class_list = [x for x in labels]
  classes = np.array(class_list)
  return classes

In [22]:
train_classes = create_label(y_train)
val_classes = create_label(y_val)
test_classes = create_label(y_test)

In [23]:
# def create_model(input_max_size, embedding_size, kernel_sizes, dropout):

#   # Input Layer
#   # URLdecode -> Unicode encode -> Give the URL converted to npumpy.darray as Input。
#   inputs = Input(shape=(input_max_size,), name='URL_input')

#   # Embedding Layer
#   x = Embedding(0xffff, embedding_size, name='Embedding')(inputs)
#   x = Reshape((input_max_size, embedding_size), name='Reshape_into_128_legnth_vector')(x)

#   # Convolution Layers
#   convolution_output = []

#   for kernel_size in kernel_sizes:
#     conv1 = Conv1D(64, kernel_size, activation='relu', padding='same', strides=1)(x)
#     pool1 = MaxPool1D(pool_size=kernel_size, padding='same', strides=1)(conv1)
#     conv2 = Conv1D(64, kernel_size, activation='relu', padding='same', strides=1)(pool1)
#     pool2 = GlobalMaxPooling1D()(conv2)
#     convolution_output.append(pool2)

#   # concat output
#   x = Concatenate(name='Concat_the_outputs')(convolution_output)

#   # reshape into 256 length vector
#   x = Reshape((256, ), name='Reshape_into_256_length_vector')(x)

#   # Fully Connected Layers
#   x = Dense(64, activation='relu', name='FullyConnectedLayer')(x)

#   # Batch Normalization
#   x = BatchNormalization()(x)

#   # DropOut
#   x = Dropout(dropout)(x)

#   # Fully Connected Layers
#   predictions = Dense(1, activation='sigmoid', name='Prediction')(x)

#   model = Model(inputs=inputs, outputs=predictions, name='Character-level_CNN')

#   return model



def create_model(input_max_size, embedding_size, kernel_sizes, dropout):

  # Input Layer
  # URLdecode -> Unicode encode -> Give the URL converted to npumpy.darray as Input。
  inputs = Input(shape=(input_max_size,), name='URL_input')

  # Embedding Layer
  x = Embedding(0xffff, embedding_size, name='Embedding')(inputs)
  x = Reshape((input_max_size, embedding_size), name='Reshape_into_128_legnth_vector')(x)

  # Convolution Layers
  convolution_output = []

  
  conv1 = Conv1D(64, kernel_size, activation='relu', padding='same', strides=1)(x)
  pool1 = MaxPool1D(pool_size=kernel_size, padding='same', strides=1)(conv1)
  conv2 = Conv1D(64, kernel_size, activation='relu', padding='same', strides=1)(pool1)
  pool2 = GlobalMaxPooling1D()(conv2)
  convolution_output.append(pool2)

  # concat output
  x = Concatenate(name='Concat_the_outputs')(convolution_output)

  # reshape into 256 length vector
  x = Reshape((64, ), name='Reshape_into_256_length_vector')(x)

  # Fully Connected Layers
  x = Dense(64, activation='relu', name='FullyConnectedLayer')(x)

  # Batch Normalization
  x = BatchNormalization()(x)

  # DropOut
  x = Dropout(dropout)(x)

  # Fully Connected Layers
  predictions = Dense(1, activation='sigmoid', name='Prediction')(x)

  model = Model(inputs=inputs, outputs=predictions, name='Character-level_CNN')

  return model  

In [24]:
# config
learning_rate = 0.001
epochs = 10

input_max_size = 1000
embedding_size = 128
kernel_sizes = [4,5,6,7]
kernel_size = 4
dropout = 0.5

loss = 'binary_crossentropy'
optimizer = 'adam'

batch_size = 128

model_filepath = 'model_dir/model'
checkpoint_filepath = 'logs/checkpoint.weights.h5'

In [25]:
from tensorflow.keras.utils import plot_model
# Tạo model
model = create_model(input_max_size, embedding_size, kernel_size, dropout)

# Vẽ hình của model
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [26]:
def train(learning_rate, input_max_size, embedding_size, kernel_sizes, dropout, loss, optimizer, train_data, train_classes, epochs, batch_size, val_data, val_classes, model_filepath):

    # optimize learning_rate
    start = learning_rate
    stop = learning_rate * 0.01
    learning_rates = np.linspace(start, stop, epochs)

    # Create model
    model = create_model(input_max_size, embedding_size, kernel_sizes, dropout)
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=['accuracy', Precision(), Recall(), 'binary_accuracy', 'categorical_accuracy'])
    
    tf_callback = TensorBoard(log_dir="logs", histogram_freq=1)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
        )


    # tranning model
    model.fit(train_data, train_classes,
              epochs=epochs,
              batch_size=batch_size,
              verbose=1,
              validation_data=(val_data, val_classes),
              callbacks=[model_checkpoint_callback]
              )

    model.save(model_filepath + '.h5')
    model.save_weights(model_filepath + '_weight.weights.h5')

In [27]:
print("Beigin Trainning")
train(learning_rate, input_max_size, embedding_size, kernel_size, dropout, loss, optimizer, train_data, train_classes, epochs, batch_size, val_data, val_classes, model_filepath)
print("End Trainning")

Beigin Trainning
Epoch 1/10
   3/3568 ━━━━━━━━━━━━━━━━━━━━ 3:34 60ms/step - accuracy: 0.6003 - binary_accuracy: 0.6003 - categorical_accuracy: 0.0000e+00 - loss: 0.6554 - precision: 0.4878 - recall: 0.6105

I0000 00:00:1712745406.865846      86 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3568/3568 ━━━━━━━━━━━━━━━━━━━━ 138s 36ms/step - accuracy: 0.9494 - binary_accuracy: 0.9494 - categorical_accuracy: 0.3960 - loss: 0.1350 - precision: 0.9095 - recall: 0.9616 - val_accuracy: 0.9676 - val_binary_accuracy: 0.9676 - val_categorical_accuracy: 0.3883 - val_loss: 0.0947 - val_precision: 0.9268 - val_recall: 0.9904
Epoch 2/10
3568/3568 ━━━━━━━━━━━━━━━━━━━━ 125s 35ms/step - accuracy: 0.9657 - binary_accuracy: 0.9657 - categorical_accuracy: 0.3573 - loss: 0.0893 - precision: 0.9368 - recall: 0.9736 - val_accuracy: 0.9700 - val_binary_accuracy: 0.9700 - val_categorical_accuracy: 0.3883 - val_loss: 0.0803 - val_precision: 0.9305 - val_recall: 0.9929
Epoch 3/10
3568/3568 ━━━━━━━━━━━━━━━━━━━━ 124s 35ms/step - accuracy: 0.9672 - binary_accuracy: 0.9672 - categorical_accuracy: 0.3810 - loss: 0.0831 - precision: 0.9386 - recall: 0.9759 - val_accuracy: 0.9702 - val_binary_accuracy: 0.9702 - val_categorical_accuracy: 0.3883 - val_loss: 0.0787 - val_precision: 0.9339 - val_recall: 0.9894


In [28]:
model_filepath = 'model_dir/model'
model = load_model(model_filepath+'.h5')

In [29]:
result = model.evaluate(test_data, test_classes, verbose=1)

# print("test_loss: ", test_loss)
# print("test_accuracy: ", test_accuracy)
# print("test_precision: ", test_precision)
# print("test_recall: ", test_recall)

print("test_loss: ", result[0])
print("test_accuracy: ", result[1])
print("test_precision: ", result[4])
print("test_recall: ", result[5])

1903/1903 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9707 - binary_accuracy: 0.9707 - categorical_accuracy: 0.3613 - loss: 0.0706 - precision: 0.9472 - recall: 0.9760
test_loss:  0.07029008865356445
test_accuracy:  0.9706476330757141
test_precision:  0.9482751488685608
test_recall:  0.9744583368301392
